<a href="https://colab.research.google.com/github/SanSan987/Final_Proj_ML_FA/blob/main/DZ3_DEn_Chubar_OO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving purchases.csv to purchases.csv
Saving products.csv to products.csv
Saving users.csv to users.csv


In [7]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz


--2025-03-11 22:24:06--  https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz’

spark-3.5.0-bin-had 100%[===================>] 381.85M  19.3MB/s    in 21s     

2025-03-11 22:24:27 (18.2 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz’ saved [400395283/400395283]



In [8]:
import os
# Налаштування змінних середовища
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Підключення findspark
!pip install findspark
import findspark
findspark.init()

# Створення SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.version


'3.5.0'

In [9]:
# Завантаження CSV-файлів
users_df = spark.read.csv("users.csv", header=True, inferSchema=True)
purchases_df = spark.read.csv("purchases.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)

In [10]:
# Очищення даних (видалення рядків із пропущеними значеннями)
users_df = users_df.dropna()
purchases_df = purchases_df.dropna()
products_df = products_df.dropna()


In [13]:
# Визначення загальної суми покупок за кожною категорією продуктів
from pyspark.sql.functions import col, sum as spark_sum, round

# Об'єднання таблиць purchases та products
purchases_products_df = purchases_df.join(products_df, "product_id")

# Додавання стовпчика total_price (кількість × ціна)
purchases_products_df = purchases_products_df.withColumn(
    "total_price", col("quantity") * col("price")
)

# Загальна сума за категорією продуктів
total_by_category_df = purchases_products_df.groupBy("category").agg(
    spark_sum("total_price").alias("total_spent")
).withColumn("total_spent", round(col("total_spent"), 2))


total_by_category_df.show()


+-----------+-----------+
|   category|total_spent|
+-----------+-----------+
|       Home|     1523.5|
|     Sports|     1802.5|
|Electronics|     1174.8|
|   Clothing|      790.3|
|     Beauty|      459.9|
+-----------+-----------+



In [17]:
# Визначення суми покупок за категорією для користувачів віком від 18 до 25 включно
# Фільтрування користувачів за віком 18-25
young_users_df = users_df.filter((col("age") >= 18) & (col("age") <= 25))

# Об'єднання всіх таблиць
young_purchases_df = purchases_products_df.join(young_users_df, "user_id")

# Сума витрат за категорією для вікової категорії 18-25
young_total_by_category_df = young_purchases_df.groupBy("category").agg(
    spark_sum("total_price").alias("total_spent")
).withColumn("total_spent", round(col("total_spent"), 2))

young_total_by_category_df.show()


+-----------+-----------+
|   category|total_spent|
+-----------+-----------+
|       Home|      361.1|
|     Sports|      310.5|
|Electronics|      249.6|
|   Clothing|      245.0|
|     Beauty|       41.4|
+-----------+-----------+



In [18]:
# Визначення частки витрат за категорією від загальних витрат для вікової групи 18-25 років
# Загальна сума витрат користувачів віком 18-25 років
total_young_spent = young_total_by_category_df.agg(
    spark_sum("total_spent").alias("overall_total")
).collect()[0]["overall_total"]

# Розрахунок відсоткової частки за категорією
percentage_by_category_df = young_total_by_category_df.withColumn(
    "percentage",
    round((col("total_spent") / total_young_spent) * 100, 2)
)

percentage_by_category_df.show()


+-----------+-----------+----------+
|   category|total_spent|percentage|
+-----------+-----------+----------+
|       Home|      361.1|      29.9|
|     Sports|      310.5|     25.71|
|Electronics|      249.6|     20.67|
|   Clothing|      245.0|     20.29|
|     Beauty|       41.4|      3.43|
+-----------+-----------+----------+



In [19]:
# Вибір трьох категорій з найвищим відсотком витрат споживачами віком від 18 до 25 років
top_3_categories_df = percentage_by_category_df.orderBy(col("percentage").desc()).limit(3)

top_3_categories_df.show()


+-----------+-----------+----------+
|   category|total_spent|percentage|
+-----------+-----------+----------+
|       Home|      361.1|      29.9|
|     Sports|      310.5|     25.71|
|Electronics|      249.6|     20.67|
+-----------+-----------+----------+

